# PLAN

- [x] Acquisition
    - [x] read the csv into a dataframe
- [ ] Preparation
    - [ ] no missing values
    - [x] drop columns that are not needed
    - [x] change case to lower case
    - [x] make sure everything has right dtype
    - [ ] normalize what needs to be normalized
    - [x] rename columns for clarification
- [ ] Exploration
    - [ ] answer ALL questions raised
        - [x] Which locations are the most frequent sites of SSO?
        - [x] Which location have the most volume of overflow?
        - [x] What are most common root causes of SSO?
        - [x] Where do the majority of overflow go?

    - [ ] visualize important findings
    - [ ] decide what TODO items to keep
- [ ] Modeling
    - [ ] predict 
- [ ] Delivery
    - [ ] report
    - [ ] prezi slides
    - [ ] website

# ENVIRONMENT

In [1]:
import os
import acquire_sso as acquire
import prepare_sso as prepare
import pandas as pd
import numpy as np

# data visualization 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm

from datetime import timedelta, datetime
from pylab import rcParams

# to explode the DataFrames and avoid truncation
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from fbprophet import Prophet

# ACQUIRE

#### _Let's read in the data from the csv file and take a peek at te first five records._

In [2]:
df = acquire.read_data('saws-sso.csv')

In [3]:
df.head()

,SSO_ID,INSPKEY,SERVNO,REPORTDATE,SPILL_ADDRESS,SPILL_ST_NAME,TOTAL_GAL,GALSRET,GAL,SPILL_START,SPILL_STOP,HRS,CAUSE,COMMENTS,ACTIONS,WATERSHED,UNITID,UNITID2,DISCHARGE_TO,DISCHARGE_ROUTE,COUNCIL_DISTRICT,FERGUSON,Month,Year,Week,EARZ_ZONE,Expr1029,PIPEDIAM,PIPELEN,PIPETYPE,INSTYEAR,DWNDPTH,UPSDPTH,Inches_No,RainFall_Less3,SPILL ADDRESS,SewerAssetExp,NUM_SPILLS_COMPKEY,NUM_SPILLS_24MOS,PREVSPILL_24MOS,UNITTYPE,ASSETTYPE,LASTCLND,ResponseTime,ResponseDTTM,Public Notice,TIMEINT,Root_Cause,STEPS_TO_PREVENT,SPILL_START_2,SPILL_STOP_2,HRS_2,GAL_2,SPILL_START_3,SPILL_STOP_3,HRS_3,GAL_3
0,6582,567722.0,NaN,3/10/19,3200,THOUSAND OAKS DR,2100,2100.0,2100.0,3/10/2019 1:16:00 PM,3/10/2019 2:40:00 PM,1.400000,Grease,Spill ContainedReturned to SystemArea Cleaned ...,CLEANED MAIN,SALADO CREEK,66918,66917,STREET,None,NaN,172A2,3,2019,11,0.0,NaN,8.0,16.55,PVC,1997.0,NaN,NaN,NaN,NaN,3200 THOUSAND OAKS DR,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,0.45,10-Mar-19,False,24.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
1,6583,567723.0,NaN,3/10/19,6804,S FLORES ST,80,0.0,80.0,3/10/2019 2:25:00 PM,3/10/2019 3:45:00 PM,1.333333,Grease,Spill ContainedArea Cleaned and Disinfected,CLEANED MAIN,DOS RIOS,24250,24193,STORMDRAIN,None,3.0,251A3,3,2019,11,0.0,NaN,8.0,157.00,PVC,1988.0,NaN,NaN,NaN,NaN,6804 S FLORES,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,1.08,10-Mar-19,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
2,6581,567714.0,NaN,3/9/19,215,AUDREY ALENE DR,79,0.0,10.0,3/9/2019 6:00:00 PM,3/9/2019 7:30:00 PM,1.500000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,DOS RIOS,2822,3351,ALLEY,None,1.0,190E4,3,2019,10,0.0,NaN,8.0,350.00,CP,1955.0,NaN,NaN,NaN,NaN,215 Audrey Alene Dr,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,1.00,09-Mar-19,False,24.0,NaN,NaN,03/10/2019 09:36,03/10/2019 10:45,1.15,69.0,NaN,NaN,0.0,0.0
3,6584,567713.0,NaN,3/9/19,3602,SE MILITARY DR,83,0.0,83.0,3/9/2019 3:37:00 PM,3/9/2019 5:00:00 PM,1.383333,Grease,Spill ContainedArea Cleaned and DisinfectedFlu...,NaN,SALADO CREEK,92804,92805,EASEMENT,None,3.0,252C3,3,2019,10,0.0,NaN,8.0,213.91,PVC,1983.0,NaN,NaN,NaN,NaN,3602 SE MILITARY DR,NaN,1,1.0,NaN,GRAVITY,Sewer Main,NaN,0.55,09-Mar-19,False,120.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0
4,6580,567432.0,NaN,3/6/19,100,PANSY LN,75,0.0,75.0,3/6/2019 9:40:00 AM,3/6/2019 9:55:00 AM,0.250000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,SALADO CREEK,61141,49543,STREET,None,2.0,192A7,3,2019,10,0.0,NaN,12.0,291.90,CP,1952.0,NaN,NaN,NaN,NaN,100 PANSY LN,NaN,2,2.0,15-Dec-18,GRAVITY,Sewer Main,NaN,0.00,06-Mar-19,False,3.0,NaN,NaN,NaN,NaN,0.00,0.0,NaN,NaN,0.0,0.0


In [4]:
df.WATERSHED.value_counts()

DOS RIOS             1572
SALADO CREEK          790
LEON CREEK            668
MEDIO CREEK           146
CCMA (Subscriber)       4
Salado Creek            1
Leon Creek              1
Dos Rios                1
Name: WATERSHED, dtype: int64

In [13]:
(df.WATERSHED == 'LEON CREEK').sum()

668

In [6]:
df[df.WATERSHED == 'Leon Creek']

,SSO_ID,INSPKEY,SERVNO,REPORTDATE,SPILL_ADDRESS,SPILL_ST_NAME,TOTAL_GAL,GALSRET,GAL,SPILL_START,SPILL_STOP,HRS,CAUSE,COMMENTS,ACTIONS,WATERSHED,UNITID,UNITID2,DISCHARGE_TO,DISCHARGE_ROUTE,COUNCIL_DISTRICT,FERGUSON,Month,Year,Week,EARZ_ZONE,Expr1029,PIPEDIAM,PIPELEN,PIPETYPE,INSTYEAR,DWNDPTH,UPSDPTH,Inches_No,RainFall_Less3,SPILL ADDRESS,SewerAssetExp,NUM_SPILLS_COMPKEY,NUM_SPILLS_24MOS,PREVSPILL_24MOS,UNITTYPE,ASSETTYPE,LASTCLND,ResponseTime,ResponseDTTM,Public Notice,TIMEINT,Root_Cause,STEPS_TO_PREVENT,SPILL_START_2,SPILL_STOP_2,HRS_2,GAL_2,SPILL_START_3,SPILL_STOP_3,HRS_3,GAL_3
1703,2795,162383.0,NaN,6/7/12,7523,OLD PEARSALL RD,124,124.0,124.0,6/7/2012 11:33:00 AM,6/7/2012 12:25:00 PM,0.866667,Grease,APPLIED HTH,CLEANED MAIN,Leon Creek,41296,41265,DRAINAGE CULVERT,None,4.0,679D4,6,2012,23,0.0,GREASE,8.0,400.0,PVC,1993.0,NaN,NaN,0.0,0.0,7523 OLD PEARSALL RD,NaN,2,NaN,NaN,GRAVITY,Sewer Main,NaN,NaN,NaN,False,NaN,GREASE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_geocoded = acquire.read_data('geocoded_saws.csv')

In [14]:
df_geocoded.sewershed.value_counts()

dos rios             1573
salado creek          791
leon creek            669
medio creek           146
ccma (subscriber)       4
Name: sewershed, dtype: int64

In [20]:
df_geocoded.Postal[df_geocoded.sewershed == 'leon creek'].value_counts()

78227    122
78242     99
78228     76
78240     54
78251     45
78238     42
78250     31
78229     29
78236     24
78249     24
78254     18
78257     17
78211     13
78221     11
78252     10
78245     10
78255      7
78237      6
78253      6
78256      6
78230      3
78224      3
78073      2
78015      2
78264      2
78219      1
78218      1
78208      1
78154      1
78244      1
78023      1
77377      1
Name: Postal, dtype: int64

In [24]:
wrong_zips = [78219, 78218, 78208, 78154, 78244, 77377]

In [37]:
temp = df_geocoded[['spill_street_address', 'Postal', 'sewershed']][df_geocoded['Postal'].isin(wrong_zips)]
temp.sort_values(by='sewershed')

,spill_street_address,Postal,sewershed
2315,"323 carson e, san antonio, tx",78208,dos rios
1816,"702 mason st, san antonio, tx",78208,dos rios
290,"4000 ih 10 e, san antonio, tx",78219,dos rios
397,"1717 rogers ave, san antonio, tx",78208,dos rios
1580,"902 grayson st e, san antonio, tx",78208,dos rios
198,"1112 muncey, san antonio, tx",78208,dos rios
2329,"1700 rogers ave, san antonio, tx",78208,dos rios
2481,"4065 ih 10 e, san antonio, tx",78219,dos rios
2209,"3099 houston st e, san antonio, tx",78219,dos rios
1817,"410 mason st, san antonio, tx",78208,dos rios


In [36]:
temp[temp['sewershed']=='leon creek']

,spill_street_address,Postal,sewershed
32,"12014 canyon rock ln, san antonio, tx",77377,leon creek
257,"10710 spirit roam, san antonio, tx",78154,leon creek
778,"29621 elkhorn ridge, san antonio, tx",78218,leon creek
796,"8400 redwing dr, san antonio, tx",78219,leon creek
1097,"4707 crystal hill, san antonio, tx",78244,leon creek
1131,"89 grey, san antonio, tx",78208,leon creek


- 32	12014 canyon rock ln, san antonio, tx	77377
    - Closest is in Spring Branch, 78070, but that's north of SAWS area, removed city and changed zip to 1

- 257	10710 spirit roam, san antonio, tx	78154
    - Changed zip code to 78254

- 778	29621 elkhorn ridge, san antonio, tx	78218
    - Changed to boerne, tx 78015

- 796	8400 redwing dr, san antonio, tx	78219
    - There is no Redwing Drive in SA, closest is north of Medina Lake... ??? removed city and changed zip to 1

- 1097	4707 crystal hill, san antonio, tx	78244
    - Changed zip code to 78238

- 1131	89 grey, san antonio, tx	78208
    - probably supposed to be 89 Grey Bluff, San Antonio, TX 78252... changed to that


# PREPARE

#### _Let's convert the column to lowercase to make them easier to work with and also rename the column names for clarity._

In [4]:
df = prepare.lowercase_and_rename(df)

In [5]:
df.head().T

,0,1,2,3,4
sso_id,6582,6583,6581,6584,6580
inspection_key,567722,567723,567714,567713,567432
service_number,NaN,NaN,NaN,NaN,NaN
report_date,3/10/19,3/10/19,3/9/19,3/9/19,3/6/19
spill_address,3200,6804,215,3602,100
spill_street_name,THOUSAND OAKS DR,S FLORES ST,AUDREY ALENE DR,SE MILITARY DR,PANSY LN
total_gallons,2100,80,79,83,75
gallons_returned,2100,0,0,0,0
gallons_1,2100,80,10,83,75
spill_start_1,3/10/2019 1:16:00 PM,3/10/2019 2:25:00 PM,3/9/2019 6:00:00 PM,3/9/2019 3:37:00 PM,3/6/2019 9:40:00 AM


#### _Let's make copies of the original dataframe before dropping some columns and rows to cover scenarios where we uncover more information about the variables._

In [6]:
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()

In [9]:
df.columns

Index(['sso_id', 'inspection_key', 'service_number', 'report_date', 'spill_address', 'spill_street_name', 'total_gallons', 'gallons_returned', 'gallons_1', 'spill_start_1', 'spill_stop_1', 'hours_1', 'cause', 'comments', 'actions', 'watershed', 'unit_id_1', 'unit_id_2', 'discharge_to', 'discharge_route', 'council_district', 'ferguson', 'month', 'year', 'week', 'edwards_zone', 'expr_1029', 'pipe_diameter', 'pipe_length', 'pipe_type', 'installation_year', 'downstream_depth', 'upstream_depth', 'inches_no', 'rainfall_less_3', 'spill_address_full', 'sewer_asset_exp', 'num_spills_compkey', 'num_spills_24mos', 'previous_spill_24mos', 'unit_type', 'asset_type', 'last_cleaned', 'response_time', 'response_datetime', 'public_notice', 'time_int', 'root_cause', 'steps_to_prevent', 'spill_start_2', 'spill_stop_2', 'hours_2', 'gallons_2', 'spill_start_3', 'spill_stop_3', 'hours_3', 'gallons_3'], dtype='object')

In [16]:
df.watershed.value_counts()

DOS RIOS             1572
SALADO CREEK          790
LEON CREEK            668
MEDIO CREEK           146
CCMA (Subscriber)       4
Leon Creek              1
Salado Creek            1
Dos Rios                1
Name: watershed, dtype: int64

In [17]:
df[df.watershed == 'Leon Creek']

,sso_id,inspection_key,service_number,report_date,spill_address,spill_street_name,total_gallons,gallons_returned,gallons_1,spill_start_1,spill_stop_1,hours_1,cause,comments,actions,watershed,unit_id_1,unit_id_2,discharge_to,discharge_route,council_district,ferguson,month,year,week,edwards_zone,expr_1029,pipe_diameter,pipe_length,pipe_type,installation_year,downstream_depth,upstream_depth,inches_no,rainfall_less_3,spill_address_full,sewer_asset_exp,num_spills_compkey,num_spills_24mos,previous_spill_24mos,unit_type,asset_type,last_cleaned,response_time,response_datetime,public_notice,time_int,root_cause,steps_to_prevent,spill_start_2,spill_stop_2,hours_2,gallons_2,spill_start_3,spill_stop_3,hours_3,gallons_3
1703,2795,162383.0,NaN,6/7/12,7523,OLD PEARSALL RD,124,124.0,124.0,6/7/2012 11:33:00 AM,6/7/2012 12:25:00 PM,0.866667,Grease,APPLIED HTH,CLEANED MAIN,Leon Creek,41296,41265,DRAINAGE CULVERT,None,4.0,679D4,6,2012,23,0.0,GREASE,8.0,400.0,PVC,1993.0,NaN,NaN,0.0,0.0,7523 OLD PEARSALL RD,NaN,2,NaN,NaN,GRAVITY,Sewer Main,NaN,NaN,NaN,False,NaN,GREASE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### _Let's prepare df1 for EDA_
0. remove columns that do not add information
0. drop the columns that are no longer needed.
0. Make new variable of whether this incident involved two or more spills within 24 hours
0. Remove redundant columns
0. Rename spill details into simpler names
0. Change column values to lowercase
0. Change address into titlecase
0. Fix the data types
0. Fill nan's
0. Change data type to datetime

In [ ]:
df1 = prepare.ready_df1(df1)
df1

In [ ]:
df1.dtypes

#### _Let's assign a variable with all numerical column names._

In [ ]:
df1_numerical_columns = list(df1.select_dtypes(include=[np.number]).columns.values)
df1_numerical_columns

#### _Let's assign a variable with all non-numerical column names._

In [ ]:
df1_non_numerical_columns = list(df1.select_dtypes(exclude=[np.number]).columns.values)
df1_non_numerical_columns

In [ ]:
df1.last_cleaned.value_counts(dropna=False)

In [ ]:
df1.head(6).T

In [ ]:
prepare.missing_values_col(df1)

# EXPLORE

#### _Looking for the repeat offenders..._

In [ ]:
df1.num_spills_24mos[df1.num_spills_24mos > 1].value_counts()

#### _Locations of the most frequent SSOs in 2 years_

In [ ]:
df1[['spill_street_address']][df1.num_spills_24mos >= 9]

#### _Total number of gallons spilled by the most frequent SSOs in 2 years_

In [ ]:
df1.total_gallons[df1.num_spills_24mos >= 9].agg('sum')

In [ ]:
df1[['spill_street_address', 'total_gallons', 'hours', 'root_cause',
     'unit_type', 'asset_type', 'last_cleaned', 'multiple_spills',
     'discharge_to', 'discharge_route']][df1.num_spills_24mos >= 9]

#### _Most common root causes of SSOs_

In [ ]:
df1.root_cause.value_counts()

- [ ] **TODO:** Find a way to flesh out the address using regex to account for typos etc.
- [ ] **TODO:** Maybe try using unit id's instead of addresses.
- [ ] **TODO:** Drill down to only the top 3-5 locations.
- [ ] **TODO:** Compare predictions between preventing SSO on the most frequents versus not preventing.
- [ ] **TODO:** What is causing the spills on these top 3-5 locations?

In [ ]:
df1.head(4).T

In [ ]:
df1[['spill_street_address', 'unit_id_1','unit_id_2', 'unit_type', 'asset_type']].head(15)

- [ ] **TODO:** Maybe we can do some kind of clustering to group problem areas.

In [ ]:
df1.unit_id_1.value_counts()[df1.unit_id_1.value_counts() > 7]

In [ ]:
df1.unit_id_2.value_counts()[df1.unit_id_2.value_counts() > 7]

In [ ]:
df1['root_cause'].value_counts()

In [ ]:
df1['spill_street_address'].value_counts()[df1.spill_street_address.value_counts() > 7]

#### _Looking for locations with most SSOs that are also caused by grease._

In [ ]:
df1.columns

In [ ]:
df1['counts'] = df1.root_cause
df1['counts'] = df1.groupby(['spill_street_address']).transform('count')
df1

#### _Below shows the most frequent SSOs that are caused by grease._

In [ ]:
df1.loc[(df1['counts'] >= 7) & (df1['root_cause'] == 'grease')]

#### _Below shows the most devastating SSOs by volume._

In [ ]:
df1[df1.total_gallons > 1500000]

In [ ]:
df1[df1.total_gallons > 1500000].shape

In [ ]:
df1.installation_year.value_counts().sort_index()

#### _Spills by installation year._

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df1[df1.installation_year < 9999].groupby('installation_year')['spill_street_address'].count())

In [ ]:
df1.year.value_counts().sort_index()

#### _Spills by year._

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df1[df1.year < 2019].groupby('year')['spill_street_address'].count())

#### _All observations grouped by month of the year._

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df1.groupby('month')['spill_street_address'].count())

#### _Colder months mean more grease clogs. Grease solidifies in colder temperatures._

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df1[(df1.root_cause == 'grease') & (df1.year < 2019)].groupby('month')['spill_street_address'].count())